In [1]:
from fishsense_api_sdk.client import Client
from fishsense_api_sdk.models.image import Image
from pathlib import Path
import asyncio
from typing import List
import numpy as np
import rawpy
import matplotlib.pyplot as plt
from skimage.util import img_as_float, img_as_ubyte
import cv2
import math
from tqdm.notebook import tqdm
from skimage.exposure import adjust_gamma, equalize_adapthist

In [2]:
URL = "http://localhost:8000"
DIVE_ID = 279

In [3]:
DATA_FOLDER = (Path("../data") / "REEF" / "data").absolute()
OUTPUT_FOLDER = (Path("../output") / "preprocess_groups_jpeg").absolute()

DATA_FOLDER.exists(), OUTPUT_FOLDER.exists()

(True, True)

In [4]:
client = Client(URL)

In [5]:
dive = await client.dives.get(dive_id=DIVE_ID)

dive

Dive(id=279, name='111323_Alligator DeepEast Drift2_FSL05', path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05', dive_datetime=datetime.datetime(2023, 11, 13, 12, 34, 8, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=5, dive_slate_id=None)

In [6]:
camera = await client.cameras.get(dive.camera_id)

camera

Camera(id=5, serial_number='BJ6C67988', name='FSL-05')

In [7]:
camera_intrinsics = await client.cameras.get_intrinsics(dive.camera_id)

camera_intrinsics

In [8]:
clusters = await client.images.get_clusters(dive_id=DIVE_ID)

len(clusters), len(clusters[0].image_ids), clusters[0].image_ids[0]

(53, 2, 78666)

In [9]:
cluster_images: List[List[Image]] = []

for cluster in clusters:
    cluster_images.append(await asyncio.gather(*[client.images.get(image_id=i) for i in cluster.image_ids]))

sum(len(c) for c in cluster_images), len(cluster_images), len(cluster_images[0]), cluster_images[0][0]

(155,
 53,
 2,
 Image(id=78666, path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130168.ORF', taken_datetime=datetime.datetime(2023, 11, 13, 12, 3, 31, tzinfo=TzInfo(0)), checksum='7252dfb0db0b855616e45607f58c1c93', is_canonical=True, dive_id=279, camera_id=5))

In [10]:
def process_raw(image_path: Path):
    with image_path.open("rb") as f:
        with rawpy.imread(f) as raw:
            img = img_as_float(
                raw.postprocess(
                    gamma=(1, 1),
                    no_auto_bright=True,
                    use_camera_wb=True,
                    output_bps=16,
                    user_flip=0,
                )
            )

            hsv = cv2.cvtColor(img_as_ubyte(img), cv2.COLOR_BGR2HSV)
            _, _, val = cv2.split(hsv)

            mid = 20
            mean = np.mean(val)
            meanLog = math.log(mean)
            midLog = math.log(mid * 255)
            gamma = midLog / meanLog
            gamma = 1 / gamma

            img = adjust_gamma(img, gamma=gamma)

            img = equalize_adapthist(img)

            return img_as_ubyte(img[:, :, ::-1])

In [11]:
def rectify(img: np.ndarray, camera_matrix: np.ndarray, distortion_coefficients: np.ndarray) -> np.ndarray:
    return cv2.undistort(
        img,
        camera_matrix,
        distortion_coefficients,
    )

In [12]:
for cluster in tqdm(cluster_images):
    for idx, image in enumerate(tqdm(cluster)):
        image_path = DATA_FOLDER / image.path
        target_path = OUTPUT_FOLDER / f"{image.checksum}.JPG"

        img = process_raw(image_path)
        img = rectify(img, camera_intrinsics.camera_matrix, camera_intrinsics.distortion_coefficients)

        height, width = img.shape[:2]

        # Define text properties
        text = f"{idx + 1}/{len(cluster)}"
        org = (width - 350, height - 75) # Bottom-right corner of the text
        fontFace = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 5
        color = (0, 0, 255) # Red color (BGR)
        thickness = 10
        lineType = cv2.LINE_AA

        # Put the text on the image
        cv2.putText(img, text, org, fontFace, fontScale, color, thickness, lineType)

        cv2.imwrite(target_path.as_posix(), img)

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]